# Welcome

# 1. Using Spark SQL find number of males and females for each class. (in “Hepatitis” data).

In [2]:
pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285387 sha256=95ec4d99bf1c79f5890cb634efc172bd58488924c427e21d2a66574aab0e5d7f
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Q1').getOrCreate()
sc = spark.sparkContext

In [5]:
df = spark.read.format('csv')\
.option('header', 'true')\
.option('inferSchema', True)\
.load('/content/hepatitis(1).csv')
# there are column headers
# column types automatically defined

In [6]:
df.show()

+---+------+-------+----------+-------+-------+--------+---------+----------+---------------+-------+-------+-------+---------+-------------+----+-------+-------+---------+-----+
|AGE|   SEX|STEROID|ANTIVIRALS|FATIGUE|MALAISE|ANOREXIA|LIVER_BIG|LIVER_FIRM|SPLEEN_PALPABLE|SPIDERS|ASCITES|VARICES|BILIRUBIN|ALK_PHOSPHATE|SGOT|ALBUMIN|PROTIME|HISTOLOGY|Class|
+---+------+-------+----------+-------+-------+--------+---------+----------+---------------+-------+-------+-------+---------+-------------+----+-------+-------+---------+-----+
| 30|  male|     no|        no|     no|     no|      no|       no|        no|             no|     no|     no|     no|        1|           85|  18|      4|      ?|       no| LIVE|
| 50|female|     no|        no|    yes|     no|      no|       no|        no|             no|     no|     no|     no|      0.9|          135|  42|    3.5|      ?|       no| LIVE|
| 78|female|    yes|        no|    yes|     no|      no|      yes|        no|             no|     no|    

In [7]:
df.toPandas()

,AGE,SEX,STEROID,ANTIVIRALS,FATIGUE,MALAISE,ANOREXIA,LIVER_BIG,LIVER_FIRM,SPLEEN_PALPABLE,SPIDERS,ASCITES,VARICES,BILIRUBIN,ALK_PHOSPHATE,SGOT,ALBUMIN,PROTIME,HISTOLOGY,Class
0,30,male,no,no,no,no,no,no,no,no,no,no,no,1,85,18,4,?,no,LIVE
1,50,female,no,no,yes,no,no,no,no,no,no,no,no,0.9,135,42,3.5,?,no,LIVE
2,78,female,yes,no,yes,no,no,yes,no,no,no,no,no,0.7,96,32,4,?,no,LIVE
3,31,female,?,yes,no,no,no,yes,no,no,no,no,no,0.7,46,52,4,80,no,LIVE
4,34,female,yes,no,no,no,no,yes,no,no,no,no,no,1,?,200,4,?,no,LIVE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150,46,female,yes,no,yes,yes,yes,yes,no,no,yes,yes,yes,7.6,?,242,3.3,50,yes,DIE
151,44,female,yes,no,yes,no,no,yes,yes,no,no,no,no,0.9,126,142,4.3,?,yes,LIVE
152,61,female,no,no,yes,yes,no,no,yes,no,yes,no,no,0.8,75,20,4.1,?,yes,LIVE
153,53,male,no,no,yes,no,no,yes,no,yes,yes,no,yes,1.5,81,19,4.1,48,yes,LIVE


In [8]:
from pyspark import SQLContext
sqlContext = SQLContext(spark)

/usr/local/lib/python3.10/dist-packages/pyspark/sql/context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [9]:
sql = df.registerTempTable('hpt')

/usr/local/lib/python3.10/dist-packages/pyspark/sql/dataframe.py:330: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [10]:
# Using Spark SQL find number of males and females for each class. (in “Hepatitis” data).

sqlContext.sql('Select sex, class, count(*) as count from hpt GROUP BY SEX,Class').show()

+------+-----+-----+
|   sex|class|count|
+------+-----+-----+
|female|  DIE|   32|
|  male| LIVE|   16|
|female| LIVE|  107|
+------+-----+-----+



In [11]:
sqlContext.sql('Select sex, class, count(*) as count from hpt GROUP BY SEX,Class').toPandas()

,sex,class,count
0,female,DIE,32
1,male,LIVE,16
2,female,LIVE,107


# 2. Find maximum and minimum ages for females. (in “Hepatitis” data).


In [12]:
sqlContext.sql('Select sex,min(age) as min_age ,max(age) as max_age from hpt where sex="female" group by sex').show()

+------+-------+-------+
|   sex|min_age|max_age|
+------+-------+-------+
|female|      7|     78|
+------+-------+-------+



In [13]:
sqlContext.sql('Select sex,min(age) as min_age ,max(age) as max_age from hpt where sex="female" group by sex').toPandas()

,sex,min_age,max_age
0,female,7,78


In [14]:
query = """
SELECT SEX, MAX(age) AS max_age, MIN(age) AS min_age
FROM hpt
group by sex
"""

sqlContext.sql(query).toPandas()

,SEX,max_age,min_age
0,female,78,7
1,male,69,20


# Spark ML Lib

task is mainly about building predictive model to predict churn using Big Data –
Spark techniques.

In [17]:
# 1. Import churn data (“churn.csv”), explore data
churn = spark.read.format('csv')\
.option('header', 'true')\
.option('inferSchema', True)\
.load('churn.csv')

In [18]:
churn.show()

+---------+----------+---------+-----------+---------+------+---+------+---------+-------------+---------+--------------+---------------+------+
|RowNumber|CustomerId|  Surname|CreditScore|Geography|Gender|Age|Tenure|  Balance|NumOfProducts|HasCrCard|IsActiveMember|EstimatedSalary|Exited|
+---------+----------+---------+-----------+---------+------+---+------+---------+-------------+---------+--------------+---------------+------+
|        1|  15634602| Hargrave|        619|   France|Female| 42|     2|      0.0|            1|        1|             1|      101348.88|     1|
|        2|  15647311|     Hill|        608|    Spain|Female| 41|     1| 83807.86|            1|        0|             1|      112542.58|     0|
|        3|  15619304|     Onio|        502|   France|Female| 42|     8| 159660.8|            3|        1|             0|      113931.57|     1|
|        4|  15701354|     Boni|        699|   France|Female| 39|     1|      0.0|            2|        0|             0|       93

In [19]:
churn.printSchema()

root
 |-- RowNumber: integer (nullable = true)
 |-- CustomerId: integer (nullable = true)
 |-- Surname: string (nullable = true)
 |-- CreditScore: integer (nullable = true)
 |-- Geography: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tenure: integer (nullable = true)
 |-- Balance: double (nullable = true)
 |-- NumOfProducts: integer (nullable = true)
 |-- HasCrCard: integer (nullable = true)
 |-- IsActiveMember: integer (nullable = true)
 |-- EstimatedSalary: double (nullable = true)
 |-- Exited: integer (nullable = true)



In [20]:
churn.toPandas().info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int32  
 1   CustomerId       10000 non-null  int32  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int32  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int32  
 7   Tenure           10000 non-null  int32  
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int32  
 10  HasCrCard        10000 non-null  int32  
 11  IsActiveMember   10000 non-null  int32  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int32  
dtypes: float64(2), int32(9), object(3)
memory usage: 742.3+ KB


In [21]:
churn.describe().show()

+-------+------------------+-----------------+-------+-----------------+---------+------+------------------+------------------+-----------------+------------------+-------------------+-------------------+-----------------+-------------------+
|summary|         RowNumber|       CustomerId|Surname|      CreditScore|Geography|Gender|               Age|            Tenure|          Balance|     NumOfProducts|          HasCrCard|     IsActiveMember|  EstimatedSalary|             Exited|
+-------+------------------+-----------------+-------+-----------------+---------+------+------------------+------------------+-----------------+------------------+-------------------+-------------------+-----------------+-------------------+
|  count|             10000|            10000|  10000|            10000|    10000| 10000|             10000|             10000|            10000|             10000|              10000|              10000|            10000|              10000|
|   mean|            5000.5|

In [22]:
churn.toPandas().describe().T

,count,mean,std,min,25%,50%,75%,max
RowNumber,10000.0,5.000500e+03,2886.895680,1.00,2500.75,5.000500e+03,7.500250e+03,10000.00
CustomerId,10000.0,1.569094e+07,71936.186123,15565701.00,15628528.25,1.569074e+07,1.575323e+07,15815690.00
CreditScore,10000.0,6.505288e+02,96.653299,350.00,584.00,6.520000e+02,7.180000e+02,850.00
Age,10000.0,3.892180e+01,10.487806,18.00,32.00,3.700000e+01,4.400000e+01,92.00
Tenure,10000.0,5.012800e+00,2.892174,0.00,3.00,5.000000e+00,7.000000e+00,10.00
Balance,10000.0,7.648589e+04,62397.405202,0.00,0.00,9.719854e+04,1.276442e+05,250898.09
NumOfProducts,10000.0,1.530200e+00,0.581654,1.00,1.00,1.000000e+00,2.000000e+00,4.00
HasCrCard,10000.0,7.055000e-01,0.455840,0.00,0.00,1.000000e+00,1.000000e+00,1.00
IsActiveMember,10000.0,5.151000e-01,0.499797,0.00,0.00,1.000000e+00,1.000000e+00,1.00
EstimatedSalary,10000.0,1.000902e+05,57510.492818,11.58,51002.11,1.001939e+05,1.493882e+05,199992.48


In [23]:
churn.select('exited').distinct().show()

+------+
|exited|
+------+
|     1|
|     0|
+------+



In [24]:
churn.dtypes

[('RowNumber', 'int'),
 ('CustomerId', 'int'),
 ('Surname', 'string'),
 ('CreditScore', 'int'),
 ('Geography', 'string'),
 ('Gender', 'string'),
 ('Age', 'int'),
 ('Tenure', 'int'),
 ('Balance', 'double'),
 ('NumOfProducts', 'int'),
 ('HasCrCard', 'int'),
 ('IsActiveMember', 'int'),
 ('EstimatedSalary', 'double'),
 ('Exited', 'int')]

In [25]:
churn.groupby('exited').count().show()

+------+-----+
|exited|count|
+------+-----+
|     1| 2037|
|     0| 7963|
+------+-----+



In [26]:
churn.groupby('exited').agg({'exited':'count'}).show()

+------+-------------+
|exited|count(exited)|
+------+-------------+
|     1|         2037|
|     0|         7963|
+------+-------------+



In [27]:
churn.toPandas().isnull().sum()

RowNumber          0
CustomerId         0
Surname            0
CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

In [28]:
churn.groupby('exited').agg({'tenure':'mean'}).show()

+------+-----------------+
|exited|      avg(tenure)|
+------+-----------------+
|     1|4.932744231713304|
|     0|5.033278914981791|
+------+-----------------+



In [29]:
churn.schema.fields

[StructField('RowNumber', IntegerType(), True),
 StructField('CustomerId', IntegerType(), True),
 StructField('Surname', StringType(), True),
 StructField('CreditScore', IntegerType(), True),
 StructField('Geography', StringType(), True),
 StructField('Gender', StringType(), True),
 StructField('Age', IntegerType(), True),
 StructField('Tenure', IntegerType(), True),
 StructField('Balance', DoubleType(), True),
 StructField('NumOfProducts', IntegerType(), True),
 StructField('HasCrCard', IntegerType(), True),
 StructField('IsActiveMember', IntegerType(), True),
 StructField('EstimatedSalary', DoubleType(), True),
 StructField('Exited', IntegerType(), True)]

# RDD is immutable (as tuple)

In [30]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType

print("Before:")
churn.show()
churn.printSchema()

new_schema = StructType([StructField('RowNumber', IntegerType(), False),
 StructField('CustomerId', IntegerType(), False),
 StructField('Surname', StringType(), False),
 StructField('CreditScore', IntegerType(), False),
 StructField('Geography', StringType(), False),
 StructField('Gender', StringType(), False),
 StructField('Age', IntegerType(), False),
 StructField('Tenure', IntegerType(), False),
 StructField('Balance', DoubleType(), False),
 StructField('NumOfProducts', IntegerType(), False),
 StructField('HasCrCard', IntegerType(), False),
 StructField('IsActiveMember', IntegerType(), False),
 StructField('EstimatedSalary', DoubleType(), False),
 StructField('Exited', IntegerType(), False)])

# Yeni DataFrame'i oluştur
new_df = spark.createDataFrame(data=churn.rdd, schema=new_schema)

# Yeni DataFrame'i göster
print("After:")
new_df.show()
new_df.printSchema()


Before:
+---------+----------+---------+-----------+---------+------+---+------+---------+-------------+---------+--------------+---------------+------+
|RowNumber|CustomerId|  Surname|CreditScore|Geography|Gender|Age|Tenure|  Balance|NumOfProducts|HasCrCard|IsActiveMember|EstimatedSalary|Exited|
+---------+----------+---------+-----------+---------+------+---+------+---------+-------------+---------+--------------+---------------+------+
|        1|  15634602| Hargrave|        619|   France|Female| 42|     2|      0.0|            1|        1|             1|      101348.88|     1|
|        2|  15647311|     Hill|        608|    Spain|Female| 41|     1| 83807.86|            1|        0|             1|      112542.58|     0|
|        3|  15619304|     Onio|        502|   France|Female| 42|     8| 159660.8|            3|        1|             0|      113931.57|     1|
|        4|  15701354|     Boni|        699|   France|Female| 39|     1|      0.0|            2|        0|             0| 

# 2. Discard unrelevant columns and select numeric columns for modelling. Define “Exited” column as a target and build Pipeline.

In [31]:
churn.show(5)

+---------+----------+--------+-----------+---------+------+---+------+---------+-------------+---------+--------------+---------------+------+
|RowNumber|CustomerId| Surname|CreditScore|Geography|Gender|Age|Tenure|  Balance|NumOfProducts|HasCrCard|IsActiveMember|EstimatedSalary|Exited|
+---------+----------+--------+-----------+---------+------+---+------+---------+-------------+---------+--------------+---------------+------+
|        1|  15634602|Hargrave|        619|   France|Female| 42|     2|      0.0|            1|        1|             1|      101348.88|     1|
|        2|  15647311|    Hill|        608|    Spain|Female| 41|     1| 83807.86|            1|        0|             1|      112542.58|     0|
|        3|  15619304|    Onio|        502|   France|Female| 42|     8| 159660.8|            3|        1|             0|      113931.57|     1|
|        4|  15701354|    Boni|        699|   France|Female| 39|     1|      0.0|            2|        0|             0|       93826.63|

In [32]:
# drop RowNumber, CustomerId, Surname
churn = churn.drop(*['RowNumber', 'CustomerId', 'Surname'])

In [33]:
churn.show(5)

+-----------+---------+------+---+------+---------+-------------+---------+--------------+---------------+------+
|CreditScore|Geography|Gender|Age|Tenure|  Balance|NumOfProducts|HasCrCard|IsActiveMember|EstimatedSalary|Exited|
+-----------+---------+------+---+------+---------+-------------+---------+--------------+---------------+------+
|        619|   France|Female| 42|     2|      0.0|            1|        1|             1|      101348.88|     1|
|        608|    Spain|Female| 41|     1| 83807.86|            1|        0|             1|      112542.58|     0|
|        502|   France|Female| 42|     8| 159660.8|            3|        1|             0|      113931.57|     1|
|        699|   France|Female| 39|     1|      0.0|            2|        0|             0|       93826.63|     0|
|        850|    Spain|Female| 43|     2|125510.82|            1|        1|             1|        79084.1|     0|
+-----------+---------+------+---+------+---------+-------------+---------+-------------

# OHE - One Hot Encoding

In [34]:
numlist = [field.name for field in churn.schema.fields\
           if (isinstance(field.dataType, IntegerType) or isinstance(field.dataType, DoubleType)) and field.name!='Exited' ]

In [35]:
# isinstance function
l = [1,2,3]
isinstance(l,list)

True

In [36]:
numlist

['CreditScore',
 'Age',
 'Tenure',
 'Balance',
 'NumOfProducts',
 'HasCrCard',
 'IsActiveMember',
 'EstimatedSalary']

In [37]:
churn.schema.fields

[StructField('CreditScore', IntegerType(), True),
 StructField('Geography', StringType(), True),
 StructField('Gender', StringType(), True),
 StructField('Age', IntegerType(), True),
 StructField('Tenure', IntegerType(), True),
 StructField('Balance', DoubleType(), True),
 StructField('NumOfProducts', IntegerType(), True),
 StructField('HasCrCard', IntegerType(), True),
 StructField('IsActiveMember', IntegerType(), True),
 StructField('EstimatedSalary', DoubleType(), True),
 StructField('Exited', IntegerType(), True)]

In [38]:
# simple way
numlist2=[]
for field in churn.schema.fields:
  if (isinstance(field.dataType, IntegerType) or isinstance(field.dataType, DoubleType)) and field.name != 'Exited':
    numlist2.append(field.name)

print(numlist2)

['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary']


In [39]:
catlist = [field.name for field in churn.schema.fields if isinstance(field.dataType, StringType) ]

In [40]:
catlist

['Geography', 'Gender']

In [41]:
catlist2 = []
for field in churn.schema.fields:
  if isinstance(field.dataType, StringType):
    catlist2.append(field.name)

print(catlist2)

['Geography', 'Gender']


In [42]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler

# StringIndexer is a function that concatenate string to each other in spark
stringindexer_stages = [StringIndexer(inputCol=c, outputCol='strindexed_' + c) for c in catlist]

In [43]:
stringindexer_stages

[StringIndexer_25797c6040d7, StringIndexer_670621a43795]

In [44]:
stringindexer_stages += [StringIndexer(inputCol='Exited', outputCol='label')]

In [45]:
stringindexer_stages

[StringIndexer_25797c6040d7,
 StringIndexer_670621a43795,
 StringIndexer_7ac6f0304027]

In [46]:
onehotencoder_stages = [OneHotEncoder(inputCol='strindexed_' + c, outputCol='onehot_' + c) for c in catlist]

In [47]:
onehotencoder_stages

[OneHotEncoder_f3339dd63ab4, OneHotEncoder_8c66825b0ada]

In [48]:
# Second type
OHE_stages = []
for c in catlist:
  OHE_stages.append(OneHotEncoder(inputCol='strindexed_' + c, outputCol='onehot_' + c))

print(OHE_stages)

[OneHotEncoder_402cc0328546, OneHotEncoder_b22c3ad732aa]


In [49]:
feature_columns = numlist + ['onehot_'+ c for c in catlist]

In [50]:
feature_columns

['CreditScore',
 'Age',
 'Tenure',
 'Balance',
 'NumOfProducts',
 'HasCrCard',
 'IsActiveMember',
 'EstimatedSalary',
 'onehot_Geography',
 'onehot_Gender']

In [51]:
vectorassambler_stage = VectorAssembler(inputCols=feature_columns, outputCol='features')

In [52]:
all_stages = stringindexer_stages + OHE_stages + [vectorassambler_stage]

In [53]:
all_stages

[StringIndexer_25797c6040d7,
 StringIndexer_670621a43795,
 StringIndexer_7ac6f0304027,
 OneHotEncoder_402cc0328546,
 OneHotEncoder_b22c3ad732aa,
 VectorAssembler_cfe4de8c5a8c]

In [54]:
from pyspark.ml import Pipeline

pipeline = Pipeline(stages = all_stages)

In [55]:
pipeline_model= pipeline.fit(churn)

In [56]:
final_columns = feature_columns + ['features', 'label']

In [57]:
churn.groupBy("Geography").count().show()
# normally it has to create 3 columns, one for each

+---------+-----+
|Geography|count|
+---------+-----+
|  Germany| 2509|
|   France| 5014|
|    Spain| 2477|
+---------+-----+



In [58]:
churn_df = pipeline_model.transform(churn).select(final_columns)

In [59]:
churn_df.show()
# Geography: 2 shows there are two value.
# If nothing written, -> Spain
# If left is 1, look at right, if zero -> France
# If left is 1, look at right, if one -> Germany

+-----------+---+------+---------+-------------+---------+--------------+---------------+----------------+-------------+--------------------+-----+
|CreditScore|Age|Tenure|  Balance|NumOfProducts|HasCrCard|IsActiveMember|EstimatedSalary|onehot_Geography|onehot_Gender|            features|label|
+-----------+---+------+---------+-------------+---------+--------------+---------------+----------------+-------------+--------------------+-----+
|        619| 42|     2|      0.0|            1|        1|             1|      101348.88|   (2,[0],[1.0])|    (1,[],[])|[619.0,42.0,2.0,0...|  1.0|
|        608| 41|     1| 83807.86|            1|        0|             1|      112542.58|       (2,[],[])|    (1,[],[])|[608.0,41.0,1.0,8...|  0.0|
|        502| 42|     8| 159660.8|            3|        1|             0|      113931.57|   (2,[0],[1.0])|    (1,[],[])|[502.0,42.0,8.0,1...|  1.0|
|        699| 39|     1|      0.0|            2|        0|             0|       93826.63|   (2,[0],[1.0])|    (1

# Train Test split

In [60]:
train, test = churn_df.randomSplit([0.8, 0.2], seed=123)

In [61]:
train.show(3)

+-----------+---+------+---------+-------------+---------+--------------+---------------+----------------+-------------+--------------------+-----+
|CreditScore|Age|Tenure|  Balance|NumOfProducts|HasCrCard|IsActiveMember|EstimatedSalary|onehot_Geography|onehot_Gender|            features|label|
+-----------+---+------+---------+-------------+---------+--------------+---------------+----------------+-------------+--------------------+-----+
|        350| 39|     0| 109733.2|            2|        0|             0|      123602.11|   (2,[1],[1.0])|(1,[0],[1.0])|[350.0,39.0,0.0,1...|  1.0|
|        350| 40|     0|111098.85|            1|        1|             1|      172321.21|   (2,[0],[1.0])|    (1,[],[])|[350.0,40.0,0.0,1...|  1.0|
|        350| 54|     1|152677.48|            1|        1|             1|      191973.49|       (2,[],[])|(1,[0],[1.0])|[350.0,54.0,1.0,1...|  1.0|
+-----------+---+------+---------+-------------+---------+--------------+---------------+----------------+------

In [62]:
test.show(3)

+-----------+---+------+---------+-------------+---------+--------------+---------------+----------------+-------------+--------------------+-----+
|CreditScore|Age|Tenure|  Balance|NumOfProducts|HasCrCard|IsActiveMember|EstimatedSalary|onehot_Geography|onehot_Gender|            features|label|
+-----------+---+------+---------+-------------+---------+--------------+---------------+----------------+-------------+--------------------+-----+
|        350| 51|    10|      0.0|            1|        1|             1|      125823.79|   (2,[0],[1.0])|(1,[0],[1.0])|[350.0,51.0,10.0,...|  1.0|
|        358| 52|     8|143542.36|            3|        1|             0|      141959.11|       (2,[],[])|    (1,[],[])|[358.0,52.0,8.0,1...|  1.0|
|        376| 29|     4|115046.74|            4|        1|             0|      119346.88|   (2,[1],[1.0])|    (1,[],[])|[376.0,29.0,4.0,1...|  1.0|
+-----------+---+------+---------+-------------+---------+--------------+---------------+----------------+------

# 3. Train your model. Use different models such as Logistic Regression, Random Forest etc.

In [63]:
from pyspark.ml.classification import  LogisticRegression, RandomForestClassifier

log_model = LogisticRegression(featuresCol='features', labelCol='label').fit(train)

In [64]:
y_pred = log_model.transform(test)

In [65]:
y_pred.show(5)

+-----------+---+------+---------+-------------+---------+--------------+---------------+----------------+-------------+--------------------+-----+--------------------+--------------------+----------+
|CreditScore|Age|Tenure|  Balance|NumOfProducts|HasCrCard|IsActiveMember|EstimatedSalary|onehot_Geography|onehot_Gender|            features|label|       rawPrediction|         probability|prediction|
+-----------+---+------+---------+-------------+---------+--------------+---------------+----------------+-------------+--------------------+-----+--------------------+--------------------+----------+
|        350| 51|    10|      0.0|            1|        1|             1|      125823.79|   (2,[0],[1.0])|(1,[0],[1.0])|[350.0,51.0,10.0,...|  1.0|[1.85105108171071...|[0.86425046479635...|       0.0|
|        358| 52|     8|143542.36|            3|        1|             0|      141959.11|       (2,[],[])|    (1,[],[])|[358.0,52.0,8.0,1...|  1.0|[-0.1145041111844...|[0.47140520802828...|       

In [66]:
y_pred.select('label', 'prediction').show()

+-----+----------+
|label|prediction|
+-----+----------+
|  1.0|       0.0|
|  1.0|       1.0|
|  1.0|       0.0|
|  1.0|       0.0|
|  1.0|       1.0|
|  1.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  1.0|       0.0|
|  0.0|       0.0|
|  1.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
+-----+----------+
only showing top 20 rows



In [67]:
y_pred.select('label', 'prediction').tail(20)

[Row(label=0.0, prediction=0.0),
 Row(label=0.0, prediction=0.0),
 Row(label=0.0, prediction=0.0),
 Row(label=0.0, prediction=0.0),
 Row(label=0.0, prediction=0.0),
 Row(label=0.0, prediction=0.0),
 Row(label=0.0, prediction=0.0),
 Row(label=0.0, prediction=0.0),
 Row(label=1.0, prediction=0.0),
 Row(label=0.0, prediction=0.0),
 Row(label=0.0, prediction=0.0),
 Row(label=0.0, prediction=0.0),
 Row(label=0.0, prediction=0.0),
 Row(label=0.0, prediction=0.0),
 Row(label=1.0, prediction=0.0),
 Row(label=0.0, prediction=1.0),
 Row(label=0.0, prediction=0.0),
 Row(label=1.0, prediction=1.0),
 Row(label=1.0, prediction=0.0),
 Row(label=1.0, prediction=1.0)]

# Evaluation of Logistic Regression

In [68]:
# accuracy manually

y_pred.filter(y_pred.label==y_pred.prediction).count()/y_pred.count()

0.8208661417322834

In [69]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator

evaluaterMulti = MulticlassClassificationEvaluator(labelCol='label',  predictionCol='prediction')

In [70]:
acc = evaluaterMulti.evaluate(y_pred, {evaluaterMulti.metricName: 'accuracy'})

In [71]:
acc

0.8208661417322834

In [72]:
w_precision =  evaluaterMulti.evaluate(y_pred, {evaluaterMulti.metricName: 'weightedPrecision'})

In [73]:
w_precision

0.803026712640557

In [74]:
w_recall =  evaluaterMulti.evaluate(y_pred, {evaluaterMulti.metricName: 'weightedRecall'})

In [75]:
w_recall

0.8208661417322834

In [76]:
f1 =  evaluaterMulti.evaluate(y_pred, {evaluaterMulti.metricName: 'f1'})

In [77]:
f1

0.7814069496233127

In [78]:
from pyspark.mllib.evaluation import MulticlassMetrics

y_pred_rdd = y_pred.select("label", "prediction").rdd.map(tuple)

metrics = MulticlassMetrics(y_pred_rdd)

confusion_matrix = metrics.confusionMatrix().toArray()

print("Confusion Matrix:")
print(confusion_matrix)

# there are a lot of zeros

# TN   FN
# FP   TP

/usr/local/lib/python3.10/dist-packages/pyspark/sql/context.py:157: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


Confusion Matrix:
[[1576.  325.]
 [  39.   92.]]


In [79]:
evaluater = BinaryClassificationEvaluator(rawPredictionCol= 'rawPrediction', metricName='areaUnderROC')

In [80]:
roc_auc = evaluater.evaluate(y_pred)

In [81]:
roc_auc

0.7660563809014725

# Random Forest Classifier

In [82]:
rf_model = RandomForestClassifier(featuresCol='features', labelCol='label').fit(train)

In [83]:
y_pred_rf = rf_model.transform(test)

In [84]:
y_pred_rf.show(5)

+-----------+---+------+---------+-------------+---------+--------------+---------------+----------------+-------------+--------------------+-----+--------------------+--------------------+----------+
|CreditScore|Age|Tenure|  Balance|NumOfProducts|HasCrCard|IsActiveMember|EstimatedSalary|onehot_Geography|onehot_Gender|            features|label|       rawPrediction|         probability|prediction|
+-----------+---+------+---------+-------------+---------+--------------+---------------+----------------+-------------+--------------------+-----+--------------------+--------------------+----------+
|        350| 51|    10|      0.0|            1|        1|             1|      125823.79|   (2,[0],[1.0])|(1,[0],[1.0])|[350.0,51.0,10.0,...|  1.0|[15.1930048436050...|[0.75965024218025...|       0.0|
|        358| 52|     8|143542.36|            3|        1|             0|      141959.11|       (2,[],[])|    (1,[],[])|[358.0,52.0,8.0,1...|  1.0|[2.63536344549608...|[0.13176817227480...|       

In [85]:
y_pred_rf.select('label', 'prediction').show()

+-----+----------+
|label|prediction|
+-----+----------+
|  1.0|       0.0|
|  1.0|       1.0|
|  1.0|       1.0|
|  1.0|       0.0|
|  1.0|       1.0|
|  1.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  1.0|       1.0|
|  0.0|       0.0|
|  1.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
+-----+----------+
only showing top 20 rows



In [86]:
y_pred_rf.select('label', 'prediction').tail(20)

[Row(label=0.0, prediction=0.0),
 Row(label=0.0, prediction=0.0),
 Row(label=0.0, prediction=0.0),
 Row(label=0.0, prediction=0.0),
 Row(label=0.0, prediction=0.0),
 Row(label=0.0, prediction=0.0),
 Row(label=0.0, prediction=0.0),
 Row(label=0.0, prediction=0.0),
 Row(label=1.0, prediction=1.0),
 Row(label=0.0, prediction=0.0),
 Row(label=0.0, prediction=0.0),
 Row(label=0.0, prediction=0.0),
 Row(label=0.0, prediction=0.0),
 Row(label=0.0, prediction=0.0),
 Row(label=1.0, prediction=0.0),
 Row(label=0.0, prediction=1.0),
 Row(label=0.0, prediction=0.0),
 Row(label=1.0, prediction=1.0),
 Row(label=1.0, prediction=1.0),
 Row(label=1.0, prediction=1.0)]

# Evaluation of Random Forest Classifier

In [87]:
# accuracy manually

y_pred_rf.filter(y_pred_rf.label==y_pred_rf.prediction).count()/y_pred_rf.count()

0.8523622047244095

In [95]:
evaluaterMulti_rf = MulticlassClassificationEvaluator(labelCol='label',  predictionCol='prediction')

In [96]:
acc_rf = evaluaterMulti_rf.evaluate(y_pred_rf, {evaluaterMulti.metricName: 'accuracy'})

In [97]:
acc_rf

0.8294559381524378

In [98]:
w_precision_rf =  evaluaterMulti_rf.evaluate(y_pred_rf, {evaluaterMulti.metricName: 'weightedPrecision'})

In [99]:
w_precision_rf

0.8294559381524378

In [100]:
w_recall_rf =  evaluaterMulti_rf.evaluate(y_pred_rf, {evaluaterMulti.metricName: 'weightedRecall'})

In [101]:
w_recall_rf

0.8294559381524378

In [102]:
f1_rf =  evaluaterMulti_rf.evaluate(y_pred_rf, {evaluaterMulti.metricName: 'f1'})

In [103]:
f1_rf

0.8294559381524378

In [104]:
y_pred_rdd_rf = y_pred_rf.select("label", "prediction").rdd.map(tuple)

metrics_rf = MulticlassMetrics(y_pred_rdd_rf)

confusion_matrix_rf = metrics_rf.confusionMatrix().toArray()

print("Confusion Matrix:")
print(confusion_matrix_rf)

# there are a lot of zeros

# TN   FN
# FP   TP

/usr/local/lib/python3.10/dist-packages/pyspark/sql/context.py:157: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


Confusion Matrix:
[[1579.  264.]
 [  36.  153.]]


In [106]:
evaluater_rf = BinaryClassificationEvaluator(rawPredictionCol= 'rawPrediction', metricName='areaUnderROC')

In [107]:
roc_auc_rf = evaluater_rf.evaluate(y_pred_rf)

In [108]:
roc_auc_rf

0.8290308929327124

# Thanks